In [1]:
me = !whoami
env = !which python
pyv = !python -V
cuda = !whereis cuda  
cuda_version = !nvcc --version
print('User: ', me)
print('Conda env: ', env)
print('Python version: ', pyv)
print('Cuda module: ', cuda)
cuda_version

User:  ['pablorr']
Conda env:  ['/n/home01/pablorr/.conda/envs/torch37/bin/python']
Python version:  ['Python 3.7.0']
Cuda module:  ['cuda:']


['nvcc: NVIDIA (R) Cuda compiler driver',
 'Copyright (c) 2005-2017 NVIDIA Corporation',
 'Built on Fri_Sep__1_21:08:03_CDT_2017',
 'Cuda compilation tools, release 9.0, V9.0.176']

In [2]:
!free -g

              total        used        free      shared  buff/cache   available
Mem:             94           3          52           0          38          89
Swap:             0           0           0


In [3]:
import os
import multiprocessing
from beautifultable import BeautifulTable as BT


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler


bl = print('\n') 
import warnings
warnings.filterwarnings("ignore")


import sys
sys.path.append('..')
from utils import load_dataset, count_parameters, figures


In [4]:
''' 
CONFIGURATION 
-------------

Catch from the parser all the parameters to define the training
'''
print('CONFIGURATION')
print('-------------')

dataset = 'fruits-small'

# Backup code to debug from python shell - no parser
save = False                # Activate results saving 
draws = False               # Activate showing the figures
testing = False             # Activate test to run few iterations per epoch       
comments = True             # Activate printing comments
createlog = False           # Activate option to save the logs in .txt
save_frequency = 1          # After how many epochs save stats
ensemble_type = 'Big'       # Single model big 
#ensemble_type = 'Huge'     # Single model huge
learning_rate = 0.1
batch_size = 128
n_iters = 64000

momentum = 0.9
weight_decay = 1e-4

n_epochs = int(n_iters / batch_size)

# GPU if CUDA is available
cuda = torch.cuda.is_available()
n_workers = multiprocessing.cpu_count()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpus = True if torch.cuda.device_count() > 1 else False
mem = False if device == 'cpu' else True

table = BT()
table.append_row(['Python Interpreter', os.system("which python")])
table.append_row(['Python Version', os.system('python -V')])
table.append_row(['Memory check (MBs)', os.system('free -m -h')])
table.append_row(['Cuda', str(cuda)])
table.append_row(['Device', str(device)])
table.append_row(['Cores', str(n_workers)])
table.append_row(['GPUs', str(torch.cuda.device_count())])
table.append_row(['CUDNN Enabled', str(torch.backends.cudnn.enabled)])
print(table)


CONFIGURATION
-------------
+--------------------+------+
| Python Interpreter |  0   |
+--------------------+------+
|   Python Version   |  0   |
+--------------------+------+
| Memory check (MBs) |  0   |
+--------------------+------+
|        Cuda        | True |
+--------------------+------+
|       Device       | cuda |
+--------------------+------+
|       Cores        |  48  |
+--------------------+------+
|        GPUs        |  2   |
+--------------------+------+
|   CUDNN Enabled    | True |
+--------------------+------+


In [5]:
'''
DEFININTION OF PATHS 
--------------------
Define all the paths to load / save files
Ensure all those paths are correctly defined before moving on
'''

print('DEFINITION OF PATHS')
print('-------------------')
scripts = os.getcwd()
root = os.path.abspath(os.path.join(scripts, '../'))
results = os.path.abspath(os.path.join(root, 'results'))
data_path = os.path.abspath(os.path.join(root, '../datasets'))

path_to_logs = os.path.join(results, 'logs', 'resnets')
path_to_models = os.path.join(results, 'models', 'resnets')
path_to_figures = os.path.join(results, 'figures', 'resnets')
path_to_dataframes = os.path.join(results, 'dataframes', 'resnets')

train_log = os.path.join(path_to_logs, 'train')
test_log = os.path.join(path_to_logs, 'test')

print('Root path: ', root)
print('Script path: ', scripts)
print('Result path: ', results)
print('DataFolder path: ', data_path)

assert os.path.exists(root), 'Root folder not found'
assert os.path.exists(scripts), 'Scripts folder not found'
assert os.path.exists(results), 'Results folder not found'
assert os.path.exists(data_path), 'Data folder not found'
assert os.path.exists(path_to_logs), 'Logs folder not found'
assert os.path.exists(path_to_models), 'Models folder not found'
assert os.path.exists(path_to_figures), 'Figure folder not found'
assert os.path.exists(path_to_dataframes), 'Dataframes folder not found'

paths = {
        'root': root, 
        'script': scripts,
        'data': data_path,
        'resulsts': results,
        'logs': {'train': train_log, 'test': test_log}, 
        'models': path_to_models,
        'figures': path_to_figures,
        'dataframes': path_to_dataframes
        }


DEFINITION OF PATHS
-------------------
Root path:  /n/home01/pablorr/Single_vs_Ensemble_of_NNs
Script path:  /n/home01/pablorr/Single_vs_Ensemble_of_NNs/ResNets
Result path:  /n/home01/pablorr/Single_vs_Ensemble_of_NNs/results
DataFolder path:  /n/home01/pablorr/datasets


In [7]:
# 1 - Import the Dataset
# ----------------------
print('IMPORTING DATA')
print('--------------')

train_set, valid_set, test_set = load_dataset(data_path, dataset, comments=comments)

train_loader = DataLoader(dataset = train_set.dataset, 
                          sampler=SubsetRandomSampler(train_set.indices),
                          batch_size = batch_size, num_workers=n_workers,
                          pin_memory = mem)

valid_loader = DataLoader(dataset = valid_set.dataset, 
                          sampler=SubsetRandomSampler(valid_set.indices),
                          batch_size = batch_size, num_workers=n_workers,
                          pin_memory = mem)

test_loader = DataLoader(dataset = test_set, batch_size = 1,
                         shuffle = False, num_workers=n_workers, pin_memory = mem)

IMPORTING DATA
--------------


AssertionError: Not found folder for this particular dataset